# Extract excitations by specifying molecule name 

The user inputs a molecule name. If the name is a valid molecule name, it is matched with an entry in the PubChem database. Example: 'water'. The corresponding SMILES code is then extracted from the database and used to compute the molecule's geometry. 

The geometry is passed to PySCFDriver to calculate the excitations.

In [1]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem
from qiskit_nature.second_q.drivers import PySCFDriver

def name_to_smiles(name):
    """Convert molecule name to SMILES using PubChem."""
    # Remove leading/trailing whitespace
    name = name.strip()
    if not name:
        raise ValueError("No molecule name entered.")
    
    try:
        res = pcp.get_compounds(name, 'name')
    except Exception as e:
        # Catch network or API errors
        raise ConnectionError(f"Failed to query PubChem: {e}")
    
    if not res:
        # No compounds found → unknown molecule
        raise ValueError(
            f"Molecule '{name}' not recognized in the SMILES database.\n"
            f"Check the molecule name here: https://pubchem.ncbi.nlm.nih.gov"
        )
    
    # Return connectivity SMILES (canonical without stereochemistry)
    return res[0].connectivity_smiles


# RDKit molecule from SMILES
user_input = input('Enter a molecule name:' )

smiles = None  # Initialize variable

# --- Try-except around the SMILES lookup ---
try:
    smiles = name_to_smiles(user_input)
except (ValueError, ConnectionError) as e:
    # Only print the error message, no traceback
    print(f"Error: {e}")

# Only proceed if 'smiles' was successfully retrieved
if smiles:
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))

    # Generate 3D coordinates
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)

    conf = mol.GetConformer()
    coords = conf.GetPositions()
    symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]
        
    geometry = [(s, coords[i].tolist()) for i, s in enumerate(symbols)]
    geometry_str = "; ".join([f"{atom} {x} {y} {z}" for atom, (x, y, z) in geometry])
    print(geometry_str)


Examples of molecules in the SMILES code
Caffeine CN1C=NC2=C1C(=O)N(C(=O)N2C)C
Benzene C1=CC=CC=C1
Water O


Enter a molecule name: hydrogen


H 0.3539999999999999 0.0 0.0; H -0.3539999999999999 0.0 0.0


[12:03:24] WARNING: not removing hydrogen atom without neighbors


## Pass the geometry to PyscfDriver in Qiskit

In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, UCC

# --- 1. Pass the geometry from the code cell above ---
driver = PySCFDriver(
    atom=geometry_str,
    basis="sto3g",
    unit=DistanceUnit.ANGSTROM
)

# --- 2. Build the electronic problem ---
problem = driver.run()

# --- 3. Instantiate UCCSD to extract excitations ---
# use 's' for singles only, 'd' for doubles only
uccsd = UCC(
    num_spatial_orbitals=problem.num_spatial_orbitals,
    num_particles=problem.num_particles,
    excitations="sd",  # singles and doubles
    qubit_mapper=JordanWignerMapper(),
)

# --- 4. Fetch excitations ---
excitations = uccsd.excitation_list

# --- 5. Print results ---
print("\n=== ALL SPECIFIED EXCITATIONS ===")
for (p, q) in excitations:
    print(f"  a†({p}) a({q})")

print("\nTotal excitations:", len(excitations))



=== ALL SPECIFIED EXCITATIONS ===
  a†((0,)) a((1,))
  a†((2,)) a((3,))
  a†((0, 2)) a((1, 3))

Total excitations: 3


In [3]:
%pip list | grep qiskit

qiskit                  2.2.3
qiskit-aer              0.17.2
qiskit-algorithms       0.4.0
qiskit-ibm-runtime      0.44.0
qiskit-nature           0.7.2
Note: you may need to restart the kernel to use updated packages.


<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This code is a part of Quantum AI Biomedical Research Lab project 
    
`Estimating molecular ground and excited state energies on quantum computers'
    
© Copyright Renata Wong, 2026.

This code is licensed under the CC BY-NC 4.0 License. You may
obtain a copy of this license in the LICENSE.txt file in the root directory
of this source tree or at https://creativecommons.org/licenses/by-nc/4.0/deed.en.

Any modifications or derivative works of this code must retain this
copyright notice, and modified files need to carry a notice indicating
that they have been altered from the originals.
</div>

<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This work was supported by the National Science and Technology Council (Taiwan) grant No. NSTC 114-2112-M-182-002-MY3 and Chang Gung Memorial Hospital grant No. BMRPL94.

This work comes with an accompanying paper titled 'Quantum circuit compilation for fermionic excitations using the Jordan-Wigner mapping'.
</div>